In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error


In [10]:
train = pd.read_csv("/kaggle/input/burnout-datathon-ieeecsmuj/train.csv")
test = pd.read_csv("/kaggle/input/burnout-datathon-ieeecsmuj/test.csv")
sample_submission = pd.read_csv("/kaggle/input/burnout-datathon-ieeecsmuj/sample_submission.csv")

print(train.head())
print(train.info())
print(train.columns)


   Unique ID  Rider_ID category_x  Circuit_Length_km  Laps  Grid_Position  \
0    1894944      2659      Moto2              4.874    22             17   
1      23438      5205      Moto2              3.875    24              7   
2     939678      7392      Moto3              5.647    25              5   
3    1196312      7894      Moto3              4.810    19              3   
4    1033899      6163     MotoGP              5.809    25             21   

   Avg_Speed_kmh Track_Condition  Humidity_% Tire_Compound_Front  ... air  \
0         264.66             Wet          61                Hard  ...  23   
1         177.56             Wet          77                Soft  ...  12   
2         317.74             Dry          87                Soft  ...  22   
3         321.82             Wet          43                Soft  ...  23   
4         239.92             Wet          47                Hard  ...  22   

  ground  starts  finishes with_points  podiums  wins  min_year  max_year 

In [11]:
print(sample_submission.columns)

Index(['Unique ID', 'Lap_Time_Seconds'], dtype='object')


In [12]:
y = train['Lap_Time_Seconds']

In [15]:
# Drop unnecessary columns
drop_cols = ['Lap_Time_Seconds', 'Unique ID', 'Rider_ID', 'rider_name', 'team_name', 'bike_name', 'shortname', 'circuit_name']
X = train.drop(columns=drop_cols, errors='ignore')
X_test = test.drop(columns=drop_cols, errors='ignore')

# Combine train+test for consistent encoding
X_all = pd.concat([X, X_test], axis=0)

# One-hot encode categoricals
X_all = pd.get_dummies(X_all)

# Split back
X = X_all.iloc[:len(train), :]
X_test = X_all.iloc[len(train):, :]

In [19]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

# Train/val split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# LightGBM dataset format
train_data = lgb.Dataset(X_train, label=y_train)
val_data = lgb.Dataset(X_val, label=y_val)

# LightGBM parameters (fast + good accuracy)
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'verbosity': -1,
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.1,
    'n_jobs': -1
}

# Train the model (10x faster than RF)
model = lgb.train(params, train_data, valid_sets=[val_data], num_boost_round=100)

# Predict
test_preds = model.predict(X_test)

# Submission
submission = pd.DataFrame({
    "Unique ID": test["Unique ID"],
    "Lap_Time_Seconds": test_preds
})
submission.to_csv("/kaggle/working/solution.csv", index=False)
print("✅ Submission saved as solution.csv")


✅ Submission saved as solution.csv


In [20]:
print(submission.isnull().sum())


Unique ID           0
Lap_Time_Seconds    0
dtype: int64


In [21]:
print(len(submission), len(test))  # Both must be equal


546874 546874


In [23]:
submission = submission.sort_values("Unique ID").reset_index(drop=True)


In [26]:
# Predict on test set
test_preds = model.predict(X_test)

# Prepare submission
submission = pd.DataFrame({
    "Unique ID": test["Unique ID"],
    "Lap_Time_Seconds": test_preds
})

submission.to_csv("/kaggle/working/submission.csv", index=False)
print("✅ Submission saved as solution.csv")


✅ Submission saved as solution.csv


In [28]:
from sklearn.preprocessing import LabelEncoder

X = train.drop(columns=['Lap_Time_Seconds', 'Unique ID'])
y = train['Lap_Time_Seconds']

# Label encode all object (string) columns
label_encoders = {}
for col in X.select_dtypes(include='object').columns:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col].astype(str))
    label_encoders[col] = le

# Train-test split
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
from lightgbm import LGBMRegressor
model = LGBMRegressor()
model.fit(X_train, y_train)

# Predict and evaluate
from sklearn.metrics import mean_squared_error
import numpy as np
val_preds = model.predict(X_val)
rmse = np.sqrt(mean_squared_error(y_val, val_preds))
print("Validation RMSE:", rmse)


Validation RMSE: 11.143313803685391
